In [1]:
#New file to vectorise the reviews to perform the data science on them

In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math


<b><h2>Handling of the review data and vectorising of the text

In [3]:
full_data = pd.read_csv("../data/review_dataset.csv")
full_data = pd.DataFrame(full_data)
full_data.dropna()
full_data = full_data.drop("Unnamed: 0", axis=1)

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train_text, X_test_text = train_test_split(full_data,test_size=0.25)
y = X_train_text["Keyword Contains"].copy()
test_y = X_test_text["Keyword Contains"].copy()
X_train_text.drop("Keyword Contains",axis=1)
X_test_text.drop("Keyword Contains",axis=1)

,Business Name,Street Number,Route,Postal Town,County,Country,Postcode,Overall Rating,Language,Rating,Review Description,Time,Date,processed review,lemmatized review,Keyword No Negative
20454,No.21 Beauty,21b,Bank Street,Dundee,Dundee City Council,United Kingdom,DD1 1RL,4.7,en,5.0,had my nails done by nikki at 21 beauty and th...,1.628579e+09,2021-08-10 07:01:17,nails done nikki 21 beauty fantastic nikki lov...,nail done nikki 21 beauty fantastic nikki lov...,False
15123,Matt's Valet,45,Spinney Close,Exeter,Devon,United Kingdom,EX2 5PE,5.0,en,5.0,changed my life,1.650921e+09,2022-04-25 21:16:13,changed life,changed life,False
12635,William H Brown Estate Agents,2,Wood Street,Wakefield,West Yorkshire,United Kingdom,WF1 2ED,4.1,en,1.0,sad to have to write this but following a very...,1.650887e+09,2022-04-25 11:51:33,sad write following poor experience feel like ...,sad write following poor experience feel like...,True
5540,Allure Style Studio,48,Upper Craigs,Stirling,Stirling,United Kingdom,FK8 2DS,5.0,en,5.0,great salon staff are lovely especially tony w...,1.603985e+09,2020-10-29 15:20:15,great salon staff lovely especially tony fanta...,great salon staff lovely especially tony fant...,False
14461,Chichester Car Wash,34,Quarry Lane,Chichester,West Sussex,United Kingdom,PO19 8ET,4.2,en,1.0,went for an exterior car wash only upon arriva...,1.648638e+09,2022-03-30 11:05:51,went exterior car wash upon arrival two vehicl...,went exterior car wash upon arrival two vehic...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20700,County Nails,22,County Road,Liverpool,Merseyside,United Kingdom,L4 3QH,4.2,en,5.0,absolutely amazing place i go everytime for my...,1.638716e+09,2021-12-05 14:53:32,absolutely amazing place go everytime nails ti...,absolutely amazing place go everytime nail ti...,False
6057,BeckyBeauty,7,Commercial Road,Shepton Mallet,Somerset,United Kingdom,BA4 5BU,5.0,en,5.0,omdsss becky is so so lovely the most down to...,1.625429e+09,2021-07-04 20:00:12,omdsss becky lovely earth person ive ever met ...,omdsss becky lovely earth person ive ever met...,False
22056,Mr Bubble Hand Carwash,82,Liverpool Road,Preston,Lancashire,United Kingdom,PR4 5SP,4.4,en,1.0,paid 50 for a full valet and bits still wasnt ...,1.651336e+09,2022-04-30 16:19:35,paid 50 full valet bits still wasnt clean prop...,paid 50 full valet bit still wasnt clean prop...,False
11757,Northwood - Southampton,1,Bellevue Road,Southampton,Southampton,United Kingdom,SO15 2AW,3.7,en,5.0,the northwoodsouthampton team have been except...,1.645099e+09,2022-02-17 12:04:39,northwoodsouthampton team exceptional proffess...,northwoodsouthampton team exceptional proffes...,False


In [6]:
count = 0
for i in X_train_text["Keyword Contains"]:
    if i == True:
        count += 1

print((count/len(X_train_text["Review Description"]))*100)

count = 0
for i in X_test_text["Keyword Contains"]:
    if i == True:
        count += 1

print((count/len(X_test_text["Review Description"]))*100)

3.9740228789323164
3.753351206434316


<h3><b>Sklearn.feature_extraction attempt to vectorise the data



In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD

In [8]:
vectoriser = TfidfVectorizer(ngram_range=(1,5))
train_tfidf = vectoriser.fit_transform(X_train_text["lemmatized review"])
test_tfidf = vectoriser.transform(X_test_text["lemmatized review"])

# full_text = vectoriser.fit_transform(full_data['lemmatized review'])
features = vectoriser.get_feature_names_out()

In [9]:
print("Train Shape: {} Rows and {} Columns".format(*train_tfidf.shape))
print("Test Shape: {} Rows and {} Columns".format(*test_tfidf.shape))

Train Shape: 16784 Rows and 1534319 Columns
Test Shape: 5595 Rows and 1534319 Columns


In [10]:
svd = TruncatedSVD(n_components=50, n_iter=20, random_state=42)
svd.fit(train_tfidf)

TruncatedSVD(n_components=50, n_iter=20, random_state=42)

In [11]:
train_tfidf = svd.transform(train_tfidf)
test_tfidf = svd.transform(test_tfidf)

In [13]:
model = LogisticRegression(multi_class='ovr',C=1)
model.fit(train_tfidf,y)

LogisticRegression(C=1, multi_class='ovr')

In [14]:
submission = model.predict(test_tfidf)
submission_df = pd.Series(submission).rename("Predicted Exploitation")
